In [112]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'
sys.path.append(pathPrefix + '/juno/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util
import get_gene_and_cohort_list_utils
import clonality_analysis_util
import math

In [44]:
facetsWhitelist = set(
['P-0000069-T01-IM3', 'P-0001248-T01-IM3', 'P-0001703-T01-IM3', 'P-0003767-T01-IM5', 'P-0004255-T01-IM5',
'P-0004260-T02-IM5', 'P-0004688-T01-IM5', 'P-0005021-T01-IM5', 'P-0006612-T01-IM5', 'P-0006753-T01-IM5',
'P-0009157-T01-IM5', 'P-0010308-T01-IM5', 'P-0010393-T02-IM6', 'P-0010499-T01-IM5', 'P-0010671-T01-IM5',
'P-0011345-T01-IM5', 'P-0011385-T01-IM5', 'P-0012397-T01-IM5', 'P-0012402-T01-IM5', 'P-0012445-T01-IM5',
'P-0012670-T01-IM5', 'P-0012726-T01-IM5', 'P-0012881-T01-IM5', 'P-0013227-T01-IM5', 'P-0013350-T01-IM5',
'P-0013400-T01-IM5', 'P-0013676-T01-IM5', 'P-0014258-T01-IM6', 'P-0014787-T01-IM6', 'P-0015885-T01-IM6',
'P-0016773-T01-IM6', 'P-0017675-T01-IM5', 'P-0017839-T01-IM6', 'P-0017925-T01-IM6', 'P-0018005-T01-IM6',
'P-0018616-T01-IM6', 'P-0019264-T01-IM6', 'P-0019360-T01-IM6', 'P-0019545-T01-IM6', 'P-0019658-T01-IM6',
'P-0019871-T01-IM6', 'P-0021090-T01-IM6', 'P-0025554-T01-IM6', 'P-0026278-T01-IM6', 'P-0026523-T01-IM6',
'P-0026962-T01-IM6', 'P-0028144-T01-IM6', 'P-0029690-T01-IM6', 'P-0029778-T01-IM6', 'P-0032113-T01-IM6',
'P-0032181-T01-IM6', 'P-0032660-T01-IM6', 'P-0033425-T01-IM6', 'P-0033605-T01-IM6', 'P-0034448-T01-IM6',
'P-0035281-T01-IM6', 'P-0036123-T01-IM6', 'P-0036500-T01-IM6', 'P-0036503-T01-IM6', 'P-0036568-T01-IM6',
'P-0036860-T01-IM6', 'P-0037288-T01-IM6', 'P-0037582-T01-IM6', 'P-0003524-T01-IM5', 'P-0004051-T01-IM5',
'P-0004865-T01-IM5', 'P-0006207-T01-IM5', 'P-0007831-T01-IM5', 'P-0007997-T01-IM5', 'P-0008345-T01-IM5',
'P-0010504-T01-IM5', 'P-0010828-T01-IM5', 'P-0011540-T01-IM5', 'P-0013537-T01-IM5', 'P-0014388-T01-IM6',
'P-0014780-T01-IM6', 'P-0016023-T01-IM6', 'P-0016099-T01-IM6', 'P-0016801-T01-IM6', 'P-0017713-T01-IM6',
'P-0019464-T01-IM6', 'P-0019649-T01-IM6', 'P-0020143-T01-IM6', 'P-0020242-T01-IM6', 'P-0020331-T01-IM6',
'P-0021077-T01-IM6', 'P-0021572-T01-IM6', 'P-0023271-T01-IM6', 'P-0023555-T01-IM6', 'P-0024633-T01-IM6',
'P-0025073-T01-IM6', 'P-0026456-T01-IM6', 'P-0027438-T01-IM6', 'P-0032496-T01-IM6', 'P-0034308-T01-IM6',
'P-0035916-T01-IM6', 'P-0000157-T01-IM3', 'P-0007843-T01-IM5', 'P-0011357-T01-IM5', 'P-0021897-T01-IM6',
'P-0020295-T01-IM6', 'P-0008646-T01-IM5', 'P-0010803-T01-IM5', 'P-0011226-T01-IM5', 'P-0013876-T01-IM5',
'P-0015288-T01-IM6', 'P-0017681-T01-IM6', 'P-0018781-T01-IM6', 'P-0019437-T01-IM6', 'P-0020151-T01-IM6',
'P-0020757-T01-IM6', 'P-0025648-T01-IM6', 'P-0027375-T01-IM6', 'P-0029228-T01-IM6', 'P-0030260-T01-IM6',
'P-0032548-T01-IM6', 'P-0032602-T01-IM6', 'P-0033410-T01-IM6', 'P-0035146-T01-IM6', 'P-0035147-T01-IM6',
'P-0006170-T01-IM5', 'P-0005197-T01-IM5', 'P-0004379-T01-IM5', 'P-0004379-T02-IM6', 'P-0011570-T01-IM5',
'P-0012171-T02-IM6', 'P-0012333-T01-IM5', 'P-0013557-T01-IM5', 'P-0016825-T01-IM6', 'P-0016972-T01-IM6',
'P-0017862-T01-IM6', 'P-0018437-T01-IM6', 'P-0020271-T01-IM6', 'P-0024488-T01-IM6', 'P-0025242-T01-IM6',
'P-0025662-T01-IM6', 'P-0029972-T01-IM6', 'P-0031195-T01-IM6', 'P-0032548-T01-IM6', 'P-0032602-T01-IM6',
'P-0035146-T01-IM6', 'P-0035147-T01-IM6', 'P-0035770-T01-IM6', 'P-0036297-T01-IM6', 'P-0009964-T02-IM5',
'P-0013787-T01-IM5', 'P-0015626-T01-IM6', 'P-0024219-T01-IM6', 'P-0032589-T01-IM6', 'P-0037227-T01-IM6',
'P-0024733-T01-IM6'
])

In [4]:
phasingData = pd.read_table(pathPrefix + '/home/ang46/lab/projects/compound_mutations/data/processed_data/data_mutations_phased_all.txt')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,45,92,93,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
allHypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=pathPrefix + '/juno/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')


In [31]:
phasingHyper = phasingData[phasingData['Tumor_Sample_Barcode'].isin(allHypermutantIds)]

In [45]:
#reload(clonality_analysis_util) #for some reason this isnt working
#whitelistFlatGenomeCases = clonality_analysis_util.get_facets_whitelist()
phasingHyper['adjPhase'] = phasingHyper.apply(lambda row: 
    'trans' if row['phase'] == 'trans or separate cells' and row['Tumor_Sample_Barcode'] in facetsWhitelist
    else row['phase'], axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [152]:
#summaryDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/phasingData.tsv', index=False, sep='\t')
plotThresh = 5
genesToHighlight = [key for key, value in dict(phasingHyper[(phasingHyper['oncogenic.1'].notnull()) & (phasingHyper['oncogenic.2'].notnull()) & (phasingHyper['adjPhase'].isin(['cis', 'trans']))][
    'Hugo_Symbol'].value_counts()).items() if value > plotThresh]

phasingHyper['label'] = phasingHyper.apply(lambda row: 
                                           '1 or 2 silent' if (row['Variant_Classification.1'] == 'Silent') | (row['Variant_Classification.2'] == 'Silent')
                                           else '1 or 2 VUS'
                                               if (not isinstance(row['oncogenic.1'], str)) | (not isinstance(row['oncogenic.2'], str))
                                           else row['Hugo_Symbol'] if row['Hugo_Symbol'] in genesToHighlight
                                           else 'other_tsg' if row['Role'] == 'TSG'
                                           else 'other_oncogene' if row['Role'] == 'Oncogene'
                                           else 'other', axis=1)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [153]:
phasingHyper['isTrans'] = phasingHyper['adjPhase'].apply(lambda x: 1 if x == 'trans' else 0 if x == 'cis' else None)
phasingHyperWrite = phasingHyper[phasingHyper['isTrans'].notnull()]

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [154]:
phasingHyperWrite.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/phasingData.tsv', index=False, sep='\t')